In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)

for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

F:\Anaconda3\envs\tf1.13.1\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Anaconda3\envs\tf1.13.1\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Anaconda3\envs\tf1.13.1\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Anaconda3\envs\tf1.13.1\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarnin

1.13.1
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)
matplotlib 3.2.2
numpy 1.18.5
pandas 1.0.5
sklearn 0.21.2
tensorflow 1.13.1
tensorflow._api.v1.keras 2.2.4-tf


In [2]:
fashion_mnist = keras.datasets.fashion_mnist

In [3]:
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()

In [4]:
x_train_all.shape

(60000, 28, 28)

In [5]:
x_test.shape

(10000, 28, 28)

In [6]:
y_train_all.shape

(60000,)

In [7]:
x_valid,x_train =  x_train_all[:5000],x_train_all[5000:]
y_valid,y_train =  y_train_all[:5000],y_train_all[5000:]

In [8]:
x_train.shape

(55000, 28, 28)

In [9]:
np.max(x_train),np.min(x_train)

(255, 0)

In [10]:
# 对数据做标准化
# x = (x - u)/std (u:均值,std:方差)

from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()

# 需要一个二维矩阵 x_train是三维的，要转成二维矩阵，再转成三维
# [None,28,28] ---> [None,784]
x_train_sacled = scalar.fit_transform(
x_train.astype(np.float32).reshape(-1,1)
).reshape(-1,28*28)

x_valid_scaled = scalar.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)

x_test_scaled = scalar.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)

In [11]:
np.max(x_train_sacled),np.min(x_train_sacled)

(2.0231433, -0.8105136)

In [12]:
hidden_units=[100,100]
class_num =10

x = tf.placeholder(dtype=tf.float32,shape=[None,28*28])
y = tf.placeholder(dtype=tf.int64,shape=[None])

input_for_next_layer = x
for hidden_unit in hidden_units:
    input_for_next_layer= tf.layers.dense(input_for_next_layer,hidden_unit,activation=tf.nn.relu)
    
logits = tf.layers.dense(input_for_next_layer,class_num)
loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=logits)

prediction = tf.argmax(logits,1)
correct_prediction = tf.equal(prediction,y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [13]:
print(x)
print(logits)

Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 10), dtype=float32)


In [14]:
# seesion
init = tf.global_variables_initializer()
batch_size = 20
epochs =10
train_steps_per_epoch = x_train.shape[0] // batch_size
valid_steps = x_valid.shape[0] // batch_size

def eval_with_sess(sess,x,y,accuracy,images,labels,batch_size):
    eval_steps = images.shape[0] // batch_size
    eval_accuracies = []
    for step in range(eval_steps):
        batch_data = images[
            step * batch_size:(step+1) * batch_size
        ]
        batch_label = labels[
            step * batch_size:(step+1) * batch_size
        ]
        accuracy_val = sess.run(accuracy,feed_dict={
            x:batch_data,
            y:batch_label
        })
            
        eval_accuracies.append(accuracy_val)
    return np.mean(eval_accuracies)

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
            batch_data = x_train_sacled[
                step * batch_size:(step+1) * batch_size
            ]
            batch_label = y_train[
                step * batch_size:(step+1) * batch_size
            ]
            loss_val,accuracy_val,_ = sess.run([loss,accuracy,train_op],feed_dict = {
                x:batch_data,
                y:batch_label
            })
            print("\r[Train] epoch : %d,step: %d,loss: %3.5f,accuracy:%2.2f" % (epoch,step,loss_val,accuracy_val))
        valid_accuracy = eval_with_sess(sess,x,y,accuracy,x_valid_scaled,y_valid,batch_size)
        print("\t[Valid] acc :%2.2f" % (valid_accuracy))

[Train] epoch : 0,step: 0,loss: 2.66161,accuracy:0.05
[Train] epoch : 0,step: 1,loss: 2.19517,accuracy:0.20
[Train] epoch : 0,step: 2,loss: 2.37998,accuracy:0.25
[Train] epoch : 0,step: 3,loss: 2.19831,accuracy:0.25
[Train] epoch : 0,step: 4,loss: 1.90337,accuracy:0.30
[Train] epoch : 0,step: 5,loss: 1.89529,accuracy:0.25
[Train] epoch : 0,step: 6,loss: 1.75834,accuracy:0.55
[Train] epoch : 0,step: 7,loss: 1.67802,accuracy:0.40
[Train] epoch : 0,step: 8,loss: 1.36350,accuracy:0.60
[Train] epoch : 0,step: 9,loss: 1.52678,accuracy:0.55
[Train] epoch : 0,step: 10,loss: 1.02921,accuracy:0.80
[Train] epoch : 0,step: 11,loss: 1.26539,accuracy:0.60
[Train] epoch : 0,step: 12,loss: 1.80574,accuracy:0.45
[Train] epoch : 0,step: 13,loss: 1.25906,accuracy:0.65
[Train] epoch : 0,step: 14,loss: 1.32196,accuracy:0.55
[Train] epoch : 0,step: 15,loss: 0.97320,accuracy:0.80
[Train] epoch : 0,step: 16,loss: 1.14338,accuracy:0.55
[Train] epoch : 0,step: 17,loss: 0.93257,accuracy:0.55
[Train] epoch : 0,st

[Train] epoch : 0,step: 173,loss: 0.21648,accuracy:0.95
[Train] epoch : 0,step: 174,loss: 0.82017,accuracy:0.65
[Train] epoch : 0,step: 175,loss: 0.57522,accuracy:0.75
[Train] epoch : 0,step: 176,loss: 0.46225,accuracy:0.80
[Train] epoch : 0,step: 177,loss: 0.49026,accuracy:0.75
[Train] epoch : 0,step: 178,loss: 0.77551,accuracy:0.75
[Train] epoch : 0,step: 179,loss: 0.82509,accuracy:0.70
[Train] epoch : 0,step: 180,loss: 0.63894,accuracy:0.80
[Train] epoch : 0,step: 181,loss: 0.62846,accuracy:0.80
[Train] epoch : 0,step: 182,loss: 0.64186,accuracy:0.75
[Train] epoch : 0,step: 183,loss: 0.48552,accuracy:0.80
[Train] epoch : 0,step: 184,loss: 0.54938,accuracy:0.85
[Train] epoch : 0,step: 185,loss: 0.66744,accuracy:0.65
[Train] epoch : 0,step: 186,loss: 0.37868,accuracy:0.85
[Train] epoch : 0,step: 187,loss: 0.54943,accuracy:0.70
[Train] epoch : 0,step: 188,loss: 0.57921,accuracy:0.85
[Train] epoch : 0,step: 189,loss: 0.61721,accuracy:0.75
[Train] epoch : 0,step: 190,loss: 0.53355,accura

[Train] epoch : 0,step: 344,loss: 0.77575,accuracy:0.75
[Train] epoch : 0,step: 345,loss: 0.38132,accuracy:0.90
[Train] epoch : 0,step: 346,loss: 0.42742,accuracy:0.80
[Train] epoch : 0,step: 347,loss: 0.37455,accuracy:0.85
[Train] epoch : 0,step: 348,loss: 0.65572,accuracy:0.75
[Train] epoch : 0,step: 349,loss: 0.71782,accuracy:0.75
[Train] epoch : 0,step: 350,loss: 0.54298,accuracy:0.80
[Train] epoch : 0,step: 351,loss: 0.33195,accuracy:0.75
[Train] epoch : 0,step: 352,loss: 0.30103,accuracy:0.90
[Train] epoch : 0,step: 353,loss: 0.44284,accuracy:0.85
[Train] epoch : 0,step: 354,loss: 0.42701,accuracy:0.75
[Train] epoch : 0,step: 355,loss: 0.34955,accuracy:0.80
[Train] epoch : 0,step: 356,loss: 0.70669,accuracy:0.70
[Train] epoch : 0,step: 357,loss: 0.73936,accuracy:0.75
[Train] epoch : 0,step: 358,loss: 0.44991,accuracy:0.90
[Train] epoch : 0,step: 359,loss: 0.95407,accuracy:0.70
[Train] epoch : 0,step: 360,loss: 0.60532,accuracy:0.80
[Train] epoch : 0,step: 361,loss: 0.59390,accura

[Train] epoch : 0,step: 517,loss: 0.78677,accuracy:0.75
[Train] epoch : 0,step: 518,loss: 0.39700,accuracy:0.90
[Train] epoch : 0,step: 519,loss: 0.47455,accuracy:0.75
[Train] epoch : 0,step: 520,loss: 0.36820,accuracy:0.80
[Train] epoch : 0,step: 521,loss: 0.51855,accuracy:0.80
[Train] epoch : 0,step: 522,loss: 0.49030,accuracy:0.95
[Train] epoch : 0,step: 523,loss: 0.88790,accuracy:0.70
[Train] epoch : 0,step: 524,loss: 0.35852,accuracy:0.85
[Train] epoch : 0,step: 525,loss: 0.21338,accuracy:0.95
[Train] epoch : 0,step: 526,loss: 0.66176,accuracy:0.75
[Train] epoch : 0,step: 527,loss: 0.34005,accuracy:0.90
[Train] epoch : 0,step: 528,loss: 0.73186,accuracy:0.65
[Train] epoch : 0,step: 529,loss: 0.49414,accuracy:0.80
[Train] epoch : 0,step: 530,loss: 0.36291,accuracy:0.90
[Train] epoch : 0,step: 531,loss: 0.39153,accuracy:0.85
[Train] epoch : 0,step: 532,loss: 0.53452,accuracy:0.80
[Train] epoch : 0,step: 533,loss: 0.53540,accuracy:0.85
[Train] epoch : 0,step: 534,loss: 0.13961,accura

[Train] epoch : 0,step: 696,loss: 0.34511,accuracy:0.90
[Train] epoch : 0,step: 697,loss: 0.77266,accuracy:0.60
[Train] epoch : 0,step: 698,loss: 0.44722,accuracy:0.90
[Train] epoch : 0,step: 699,loss: 0.25120,accuracy:0.90
[Train] epoch : 0,step: 700,loss: 0.98001,accuracy:0.60
[Train] epoch : 0,step: 701,loss: 0.42874,accuracy:0.85
[Train] epoch : 0,step: 702,loss: 0.44535,accuracy:0.80
[Train] epoch : 0,step: 703,loss: 0.71958,accuracy:0.75
[Train] epoch : 0,step: 704,loss: 1.28353,accuracy:0.55
[Train] epoch : 0,step: 705,loss: 0.57093,accuracy:0.80
[Train] epoch : 0,step: 706,loss: 0.63052,accuracy:0.70
[Train] epoch : 0,step: 707,loss: 0.31529,accuracy:0.90
[Train] epoch : 0,step: 708,loss: 0.71536,accuracy:0.70
[Train] epoch : 0,step: 709,loss: 0.27029,accuracy:0.90
[Train] epoch : 0,step: 710,loss: 0.36181,accuracy:0.85
[Train] epoch : 0,step: 711,loss: 0.59893,accuracy:0.70
[Train] epoch : 0,step: 712,loss: 0.62942,accuracy:0.75
[Train] epoch : 0,step: 713,loss: 0.67080,accura

...

[Train] epoch : 9,step: 2006,loss: 0.21405,accuracy:0.90
[Train] epoch : 9,step: 2007,loss: 0.16892,accuracy:0.95
[Train] epoch : 9,step: 2008,loss: 0.24773,accuracy:0.85
[Train] epoch : 9,step: 2009,loss: 0.25993,accuracy:0.90
[Train] epoch : 9,step: 2010,loss: 0.22152,accuracy:0.85
[Train] epoch : 9,step: 2011,loss: 0.29666,accuracy:0.85
[Train] epoch : 9,step: 2012,loss: 0.31870,accuracy:0.80
[Train] epoch : 9,step: 2013,loss: 0.14077,accuracy:1.00
[Train] epoch : 9,step: 2014,loss: 0.27666,accuracy:0.85
[Train] epoch : 9,step: 2015,loss: 0.11698,accuracy:1.00
[Train] epoch : 9,step: 2016,loss: 0.14864,accuracy:0.95
[Train] epoch : 9,step: 2017,loss: 0.30004,accuracy:0.90
[Train] epoch : 9,step: 2018,loss: 0.13780,accuracy:0.95
[Train] epoch : 9,step: 2019,loss: 0.01505,accuracy:1.00
[Train] epoch : 9,step: 2020,loss: 0.19752,accuracy:0.90
[Train] epoch : 9,step: 2021,loss: 0.12156,accuracy:0.95
[Train] epoch : 9,step: 2022,loss: 0.14661,accuracy:1.00
[Train] epoch : 9,step: 2023,lo

[Train] epoch : 9,step: 2206,loss: 0.04040,accuracy:1.00
[Train] epoch : 9,step: 2207,loss: 0.12425,accuracy:0.95
[Train] epoch : 9,step: 2208,loss: 0.13515,accuracy:0.95
[Train] epoch : 9,step: 2209,loss: 0.18725,accuracy:0.95
[Train] epoch : 9,step: 2210,loss: 0.19696,accuracy:0.90
[Train] epoch : 9,step: 2211,loss: 0.13009,accuracy:0.95
[Train] epoch : 9,step: 2212,loss: 0.32446,accuracy:0.85
[Train] epoch : 9,step: 2213,loss: 0.06084,accuracy:1.00
[Train] epoch : 9,step: 2214,loss: 0.15073,accuracy:0.95
[Train] epoch : 9,step: 2215,loss: 0.08651,accuracy:0.95
[Train] epoch : 9,step: 2216,loss: 0.35676,accuracy:0.85
[Train] epoch : 9,step: 2217,loss: 0.18134,accuracy:0.95
[Train] epoch : 9,step: 2218,loss: 0.11611,accuracy:0.90
[Train] epoch : 9,step: 2219,loss: 0.28677,accuracy:0.90
[Train] epoch : 9,step: 2220,loss: 0.31635,accuracy:0.80
[Train] epoch : 9,step: 2221,loss: 0.50455,accuracy:0.90
[Train] epoch : 9,step: 2222,loss: 0.12483,accuracy:0.90
[Train] epoch : 9,step: 2223,lo

[Train] epoch : 9,step: 2403,loss: 0.17086,accuracy:0.95
[Train] epoch : 9,step: 2404,loss: 0.21641,accuracy:0.95
[Train] epoch : 9,step: 2405,loss: 0.33063,accuracy:0.85
[Train] epoch : 9,step: 2406,loss: 0.03689,accuracy:1.00
[Train] epoch : 9,step: 2407,loss: 0.07026,accuracy:1.00
[Train] epoch : 9,step: 2408,loss: 0.24600,accuracy:0.90
[Train] epoch : 9,step: 2409,loss: 0.05429,accuracy:1.00
[Train] epoch : 9,step: 2410,loss: 0.13029,accuracy:0.95
[Train] epoch : 9,step: 2411,loss: 0.06406,accuracy:1.00
[Train] epoch : 9,step: 2412,loss: 0.55208,accuracy:0.85
[Train] epoch : 9,step: 2413,loss: 0.24583,accuracy:0.95
[Train] epoch : 9,step: 2414,loss: 0.49211,accuracy:0.85
[Train] epoch : 9,step: 2415,loss: 0.40580,accuracy:0.70
[Train] epoch : 9,step: 2416,loss: 0.16958,accuracy:0.95
[Train] epoch : 9,step: 2417,loss: 0.01853,accuracy:1.00
[Train] epoch : 9,step: 2418,loss: 0.23758,accuracy:0.90
[Train] epoch : 9,step: 2419,loss: 0.18983,accuracy:0.90
[Train] epoch : 9,step: 2420,lo

[Train] epoch : 9,step: 2606,loss: 0.11522,accuracy:0.95
[Train] epoch : 9,step: 2607,loss: 0.25514,accuracy:0.85
[Train] epoch : 9,step: 2608,loss: 0.75901,accuracy:0.80
[Train] epoch : 9,step: 2609,loss: 0.28350,accuracy:0.85
[Train] epoch : 9,step: 2610,loss: 0.51405,accuracy:0.75
[Train] epoch : 9,step: 2611,loss: 0.07010,accuracy:1.00
[Train] epoch : 9,step: 2612,loss: 0.09882,accuracy:0.95
[Train] epoch : 9,step: 2613,loss: 0.10122,accuracy:0.95
[Train] epoch : 9,step: 2614,loss: 0.17155,accuracy:0.95
[Train] epoch : 9,step: 2615,loss: 0.12856,accuracy:0.90
[Train] epoch : 9,step: 2616,loss: 0.09967,accuracy:0.95
[Train] epoch : 9,step: 2617,loss: 0.29427,accuracy:0.85
[Train] epoch : 9,step: 2618,loss: 0.05635,accuracy:1.00
[Train] epoch : 9,step: 2619,loss: 0.33350,accuracy:0.80
[Train] epoch : 9,step: 2620,loss: 0.28487,accuracy:0.90
[Train] epoch : 9,step: 2621,loss: 0.07247,accuracy:1.00
[Train] epoch : 9,step: 2622,loss: 0.18204,accuracy:0.90
[Train] epoch : 9,step: 2623,lo

	[Valid] acc :0.88
